In [27]:
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.sparse as sparse
import sklearn.gaussian_process.kernels as kernels
from KoLesky.maxheap import Heap
from KoLesky.ordering import reverse_maximin
from KoLesky.ordering import sparsity_pattern

In [3]:
def create_points(n):
    points = np.zeros((n * n, 2))
    for i in range(n):
        for j in range(n):
            perturbation = np.random.uniform(-0.2, 0.2, 2)
            points[i * n + j] = np.array([i - n/2, j - n/2]) + perturbation
    return points

Methods to calculate KL-Divergence. When the optimal cholesky factor is used, the trace factor cancels.

In [4]:
def logdet_chol(A):
    return 2 * np.sum(np.log(A.diagonal()))

def kl_div(A, B):
    n = A.shape[0]
    return 0.5 * (np.trace(np.linalg.solve(B, A)) - n + np.linalg.slogdet(B)[1] - np.linalg.slogdet(A)[1])

def sparse_kl_div(A, L):
    n = A.shape[0]
    return 0.5 * (-logdet_chol(L) - np.linalg.slogdet(A)[1])

Orderings for the Cholesky factorization by picking the maximum of the minimum distances between points.

In [5]:
def py_reverse_maximin(points):
    n = len(points)
    indices = np.zeros(n, dtype=np.int64)
    lengths = np.zeros(n)
    tree = KDTree(points)
    dists = np.linalg.norm(points - points[0], axis=1)
    indices = np.zeros(n, dtype=np.int64)
    lengths = np.zeros(n)
    heap = Heap(dists, np.arange(n))
    for i in range(n - 1, -1, -1):
        lk, k = heap.pop()
        indices[i] = k
        lengths[i] = lk
        js = tree.query_ball_point(points[k], lk)
        dists = np.linalg.norm(points[js] - points[k], axis=1)
        for index, j in enumerate(js):
            heap.decrease_key(j, dists[index])
    return indices, lengths

In [6]:
%%latex
Sparsity patterns which can naively be calculated by checking if $||x_i-x_j||_2 \leq \rho\min(l_i,l_j)$

<IPython.core.display.Latex object>

In [7]:
def naive_sparsity_pattern(points, lengths, rho):
    n = len(points)
    sparsity = {i : [] for i in range(n)}
    for i in range(n):
        for j in range(i, n):
            if np.linalg.norm(points[i] - points[j]) <= min(lengths[i], lengths[j]) * rho:
                sparsity[i].append(j)
    return sparsity

def kd_sparsity_pattern(points, lengths, rho):
    tree = KDTree(points)
    near = tree.query_ball_point(points, rho * lengths)
    return {i: [j for j in near[i] if j >= i] for i in range(len(points))}

def py_sparsity_pattern(points, lengths, rho):
    tree, offset, length_scale = KDTree(points), 0, lengths[0]
    sparsity = {}
    for i in range(len(points)):
        if lengths[i] > 2 * length_scale:
            tree, offset, length_scale = KDTree(points[i:]), i, lengths[i]
        sparsity[i] = [
            offset + j
            for j in tree.query_ball_point(points[i], rho * lengths[i])
            if offset + j >= i
        ]
    return sparsity

In [8]:
%%latex
Aggregate sparsity pattern into supernode groups such that: $l_j \leq \lambda l_i$

<IPython.core.display.Latex object>

In [9]:
def supernodes(sparsity, lengths, lamb):
    groups = []
    candidates = set(range(len(lengths)))
    agg_sparsity = {}
    i = 0
    while len(candidates) > 0:
        while i not in candidates:
            i += 1
        group = sorted(j for j in sparsity[i] if lengths[j] <= lamb * lengths[i] and j in candidates)
        groups.append(group)
        candidates -= set(group)
        s = sorted({k for j in group for k in sparsity[j]})
        agg_sparsity[group[0]] = s
        positions = {k: j for j, k in enumerate(s)}
        for j in group[1:]:
            agg_sparsity[j] = np.empty(len(s) - positions[j], dtype=int)
    return groups, agg_sparsity

In [10]:
%%latex
Naive sparse cholesky factorization using KL-Divergence by using $$L_{s_i} = \frac{\Theta_{s_i,s_i}^{-1}e_1}{\sqrt{e_1^T\Theta_{s_i,s_i}^{-1}e_1}}$$

<IPython.core.display.Latex object>

In [11]:
def col(theta):
    m = np.linalg.inv(theta)
    return m[:, 0] / np.sqrt(m[0, 0])

def chol(points, kernel, sparsity):
    n = len(points)
    ptr = np.cumsum([0] + [len(sparsity[i]) for i in range(n)])
    data, indices = np.zeros(ptr[-1]), np.zeros(ptr[-1])
    for i in range(n):
        s = sorted(sparsity[i])
        c = col(kernel(points[s]))
        data[ptr[i] : ptr[i + 1]] = c
        indices[ptr[i] : ptr[i + 1]] = s
    return sparse.csc_matrix((data, indices, ptr), shape=(n, n))

def naive_kl_cholesky(points, rho):
    indices, lengths = reverse_maximin(points)
    ordered_points = points[indices]
    sparsity = kd_sparsity_pattern(ordered_points, lengths, rho)
    kernel = kernels.Matern(length_scale=1.0, nu=0.5)
    return chol(ordered_points, kernel, sparsity)

In [12]:
%%latex
The aggregated cholesky factorization forms supernodes and then calculates the cholesky factor using $$L_{:, k} = U^{-T}e_k$$
Note that $\text{flip}(A) = PAP$ where $P$ is the order reversing permutation matrix, so $(P\text{chol}(P\Theta P)P)^T = (\text{chol}(\Theta^{-1}))^{-1}=U^T$

<IPython.core.display.Latex object>

In [13]:
def cols(theta):
    return np.flip(np.linalg.cholesky(np.flip(theta))).T

def aggregate_chol(points, kernel, sparsity, groups):
    n = len(points)
    ptr = np.cumsum([0] + [len(sparsity[i]) for i in range(n)])
    data, indices = np.zeros(ptr[-1]), np.zeros(ptr[-1])
    for group in groups:
        s = sorted(sparsity[group[0]])
        positions = {i: k for k, i in enumerate(s)}
        L_group = cols(kernel(points[s]))
        for i in group:
            k = positions[i]
            e_k = np.zeros(len(s))
            e_k[k] = 1
            col = scipy.linalg.solve_triangular(L_group, e_k, lower=True, check_finite=False)
            data[ptr[i] : ptr[i + 1]] = col[k:]
            indices[ptr[i] : ptr[i + 1]] = s[k:]
    return sparse.csc_matrix((data, indices, ptr), shape=(n, n))

def aggregated_kl_cholesky(points, rho, lamb):
    indices, lengths = reverse_maximin(points)
    ordered_points = points[indices]
    sparsity = naive_sparsity_pattern(ordered_points, lengths, rho)
    groups, agg_sparsity = supernodes(sparsity, lengths, lamb)
    kernel = kernels.Matern(length_scale=1.0, nu=0.5)
    return aggregate_chol(ordered_points, kernel, agg_sparsity, groups)

In [14]:
%%latex
The iterative method finds $L$ using the above methods, creates a new kernel matrix $L^T\Theta L$ and then finds the sparse cholesky factor $L'$. The final result is $LL'$

<IPython.core.display.Latex object>

In [15]:
def new_kernel_idx(points, kernel, L, r, c):
    # calculate L.T @ theta @ L index at r and c
    left_row = L.T[r]
    right_col = L[:, c]
    left_indices = left_row.indices
    right_indices = right_col.indices
    left_points = points[left_indices]
    right_points = points[right_indices]
    kernel_matrix = kernel(left_points, right_points)
    result = np.dot(left_row.data, np.dot(kernel_matrix, right_col.data))
    return result

def iter_col(points, kernel, s, L, calculated_entries):
    m = np.zeros((len(s), len(s)))
    for i in range(len(s)):
        for j in range(len(s)):
            if (i, j) in calculated_entries: 
                m[i, j] = calculated_entries[(i, j)]
            elif (j, i) in calculated_entries: 
                m[i, j] = calculated_entries[(j, i)]
            else: 
                m[i, j] = new_kernel_idx(points, kernel, L, s[i], s[j])
                calculated_entries[(i, j)] = m[i, j]
    return col(m)

def iter_chol(points, kernel, sparsity, L):
    n = len(points)
    ptr = np.cumsum([0] + [len(sparsity[i]) for i in range(n)])
    data, indices = np.zeros(ptr[-1]), np.zeros(ptr[-1])
    for i in range(n):
        s = sorted(sparsity[i])
        entries = {}
        c = iter_col(points, kernel, s, L, entries)
        data[ptr[i] : ptr[i + 1]] = c
        indices[ptr[i] : ptr[i + 1]] = s
    return sparse.csc_matrix((data, indices, ptr), shape=(n, n))

def naive_iterative_kl_cholesky(points, rho):
    indices, lengths = reverse_maximin(points)
    ordered_points = points[indices]
    sparsity = naive_sparsity_pattern(ordered_points, lengths, rho)
    kernel = kernels.Matern(length_scale=1.0, nu=0.5)
    L = chol(ordered_points, kernel, sparsity)
    iter_L = iter_chol(ordered_points, kernel, sparsity, L)
    return L, iter_L

def iter_cols(points, kernel, sparsity, L):
    m = np.zeros((len(sparsity), len(sparsity)))
    for i in range(len(sparsity)):
        for j in range(len(sparsity)):
            m[i, j] = new_kernel_idx(points, kernel, L, i, j)
    return cols(m)

def aggregate_iter_chol(points, kernel, sparsity, groups, L):
    n = len(points)
    ptr = np.cumsum([0] + [len(sparsity[i]) for i in range(n)])
    data, indices = np.zeros(ptr[-1]), np.zeros(ptr[-1])
    for group in groups:
        s = sorted(sparsity[group[0]])
        positions = {i: k for k, i in enumerate(s)}
        L_group = iter_cols(points, kernel, s, L)
        for i in group:
            k = positions[i]
            e_k = np.zeros(len(s))
            e_k[k] = 1
            col = scipy.linalg.solve_triangular(L_group, e_k, lower=True, check_finite=False)
            data[ptr[i] : ptr[i + 1]] = col[k:]
            indices[ptr[i] : ptr[i + 1]] = s[k:]
    return sparse.csc_matrix((data, indices, ptr), shape=(n, n))

def iterative_aggregated_kl_cholesky(points, rho, lamb):
    indices, lengths = reverse_maximin(points)
    ordered_points = points[indices]
    sparsity = naive_sparsity_pattern(ordered_points, lengths, rho)
    groups, agg_sparsity = supernodes(sparsity, lengths, lamb)
    kernel = kernels.Matern(length_scale=1.0, nu=0.5)
    theta = kernel(ordered_points)
    L = aggregate_chol(theta, agg_sparsity, groups)
    new_kernel = L.T @ theta @ L
    return L @ aggregate_chol(new_kernel, agg_sparsity, groups)

In [16]:
points = create_points(1000)
order, lengths = reverse_maximin(points)
ordered_points = points[order]

In [25]:
%timeit sparsity_pattern(ordered_points, lengths, 3.0)

10.9 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit py_sparsity_pattern(ordered_points, lengths, 3.0)

11.3 s ± 85.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
sparsity = sparsity_pattern(ordered_points, lengths, 3.0)

%timeit supernodes(sparsity, lengths, 1.5)

3.76 s ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
